# Práctica Grafos

In [1]:
import graphframes as gf
from pyspark.sql.types import StructType, StructField, StringType, LongType
import pyspark.sql.functions as F
import matplotlib.pyplot as plt

In [6]:
spark.sparkContext.setCheckpointDir('/vagrant/IPNB/checkpoints')

### 1. Cree un grafo de GraphFrames correspondiente a los ficheros de datos suministrados, en el que cada enlace es una relación de descendencia.

In [2]:
df_edges = spark.read.csv('edges-subset.csv', inferSchema=True, header=True)
df_nodes = spark.read.csv('vertices-subset.csv', inferSchema=True, header=True)
graph = gf.GraphFrame(df_nodes, df_edges)

### 2 Contenga las operaciones necesarias para contestar a las siguientes preguntas sobre el grafo creado: 
 **2.1 ¿Cuántos árboles genealógicos distintos contiene el dataset proporcionado, y qué tamaño (número de personas) tienen? Nota: definimos un árbol genealógico como el subgrafo que contiene todos sus nodos conectados y no tiene ninguna conexión a otros nodos. Tomando solo el árbol más grande, ¿cuántos nodos tiene? ¿Qué intervalo temporal abarca?**

Para encontrar el número de árboles genealógicos se asume que los miembros de las familias no se han mezclado. En tal caso, para encontrar el número de familias basta calcular el número de componentes conexas del grafo.

In [7]:
families = graph.connectedComponents()

In [8]:
num_families = families.select(F.countDistinct('component')).toPandas() # numero de componentes conexas
num_families

,count(DISTINCT component)
0,3


Se observa que hay 3 componentes conexas en el grafo, es decir, **hay 3 familias o arboles genealógicos.**

Por otro lado, para ver como de grandes son, se hace un recuento de nodos en el grafo agregando por la componente a la cual pertencen.

In [9]:
families.groupBy('component').count().toPandas()

,component,count
0,24553,1488
1,55143,1658
2,225398,1381


Se ve que **la familia más grande tiene 1658 nodos (personas).**

Para calcular el intervalo temporal, usamos la función describe sobre la columna del año de nacimiento.

In [10]:
families.filter('birth_year != "*" and component == 55143').select('birth_year').describe().toPandas()

,summary,birth_year
0,count,1457
1,mean,1707.4351407000686
2,stddev,62.76035946620318
3,min,1520
4,max,1952


Se observa que **abarca desde la generación nacida a principios del S.XVI (1520) hasta bien entrado el S.XX (1952)**

**2.2 ¿Cuántos ejemplos hay en el dataset de 3 personas de generaciones sucesivas (padre o madre, hijo o hija, nieto o nieta) en las que cada una tiene un país de nacimiento distinto (es decir, 3 países de nacimiento distintos)? Nota: no cuente para este cálculo las personas con país de nacimiento desconocido.**

Para este ejercicio filtramos previamente los valores de la columna "país de nacimiento" para eliminar los valores perdidos. Despues, se calculan todos los caminos con longitud igual 2 con los 3 nodos distintos. Finalmente, se hace un recuento del número total de caminos, lo cual es equivalente al número de ejemplos pedido.

In [11]:
df_nodes_kc = df_nodes.filter("birth_location_country != '*'")
graph_known_country = gf.GraphFrame(df_nodes_kc, df_edges)

pattern = graph_known_country.find("(a)-[e1]->(b); (b)-[e2]->(c)") # caminos con 3 nodos distintos
result = pattern.filter((pattern['a'].birth_location_country != pattern['b'].birth_location_country) &
                        (pattern['b'].birth_location_country != pattern['c'].birth_location_country))

result.count()

85

Se observa que **el número de ejemplos de 3 generaciones sucesivas nacidas en países distintos es 85.**

 **2.3 Encuentre la persona o personas del dataset con el máximo de descendencia directa (i.e.
mayor número de hijos+hijas). ¿En qué año nació? ¿Qué nacionalidad tiene? ¿Cuántos
hijos+hijas tuvo?**

Para este ejercicio se calcula el grado exterior de cada nodo del grafo y se extrae el máximo.

In [30]:
max_degree = int(graph.outDegrees.groupBy().max('outDegree').toPandas().iloc[0])
max_degree

16

Dentro del conjunto de datos, **el número máximo de descendencia directa es igual a 16 hijos+hijas.**

Despues, para encontrar información de los nodos, se filtra el conjunto de vertices y grado para obtener los Id.

In [31]:
max_desc_node_id = graph.outDegrees.filter(f"outDegree == {max_degree}").select('id')
nodes_id = max_desc_node_id.toPandas()['id'].tolist() # nodos con el max numero de descendencia

result_max_nodes_desc = graph.vertices.filter(graph.vertices['id'].isin(nodes_id))

result_max_nodes_desc.toPandas()

,id,gender,is_alive,birth_year,birth_location_city,birth_location_country,death_year,death_location_country,cause_of_death
0,43475617,female,0,1704,Isfield,England,1780,England,*
1,43871304,male,0,1700,Isfield,England,1767,England,*


Se observa que **las dos personas con mayor número de descendencia directa nacieron en 1704 y 1700, y eran de Inglaterra.**

Para calcular la cantidad de hijos e hijas, se obtienen sus nodos vecinos y despues se hace un recuento de observaciones agregando por el id de los padres y el genero de los hijos.

In [24]:
pattern_path1 = graph.find("(a)-[e1]->(b)")
neigs = pattern_path1.filter(pattern_path1['a'].id.isin(nodes_id))
neigs.groupBy(neigs.a.id, neigs.b.gender).count().collect()

[Row(a[id]=43871304, b[gender]='female', count=7),
 Row(a[id]=43475617, b[gender]='male', count=9),
 Row(a[id]=43475617, b[gender]='female', count=7),
 Row(a[id]=43871304, b[gender]='male', count=9)]

Se observa que en ambos casos se tuvieron 7 hijas y 9 hijos. Por los datos de los nodos (nacimiento, país, número de hijos y sexo) parece que fueron pareja.

**2.4 Cree ahora la lista de personas que tengan el mayor número de descendientes +
ascendientes directos (de primer nivel), es decir, la suma del número de descendientes y
del número de ascendientes directos encontrados en el dataset para esas personas sea
la mayor posible. Muestre y comente los dos primeros resultados de la lista encontrada.**

Se repite el proceso anterior pero esta vez sobre el dataframe que contiene la suma de los vecinos interiores y exteriores a cada nodo, sumando ambos para obtener el grado de cada nodo.

In [33]:
total_degrees = graph.outDegrees.join(graph.inDegrees, 'id')
total_degrees = total_degrees.withColumn('totalDegree',total_degrees.inDegree+total_degrees.outDegree) 

In [34]:
max_degree_total = int(total_degrees.groupBy().max('totalDegree').toPandas().iloc[0])
max_degree_total

18

In [35]:
max_degree_total # grado máximo 

18

In [39]:
max_desc_node_id = total_degrees.filter(f"totalDegree == {max_degree_total}").select('id')
nodes_id_total = max_desc_node_id.toPandas()['id'].tolist() # nodos con el max numero de descendencia

result_max_nodes_desc = graph.vertices.filter(graph.vertices['id'].isin(nodes_id_total))

result_max_nodes_desc.toPandas()

,id,gender,is_alive,birth_year,birth_location_city,birth_location_country,death_year,death_location_country,cause_of_death
0,43475617,female,0,1704,Isfield,England,1780,England,*


El grado total máximo es 18, y el grado interior máximo es 2, lo cual es consistente con que cada persona debe tener, a lo sumo, una ascendencia directa de dos personas (padre y madre) salvo datos faltantes.

En este caso, la persona con más ascendencia y descendencia directa es la misma mujer (mismo Id) que tuvo 16 hijos en el apartado anterior. Sin embargo, el hombre ahora no aparece. **Dado que el hombre no aparece en el resultado de grado interior, la causa del resultado de este apartado es que es la mujer la que forma parte del tronco familiar y el hombre no tiene ascendencia directa dentro del árbol genealógico, ya que proviene de una familia distinta.**

In [40]:
graph.inDegrees.filter(graph.inDegrees['id'].isin(nodes_id)).collect() # id del nodo con grado interior maximo

[Row(id=43475617, inDegree=2)]